## 1. Analysis on the Key length

* it's certain that offset is 6 and the key length is 24
* from known plaintext, it can be inferred that the following 6 letters are 'ucture'
* So, I've can get some candidates of keys with length 5
* and kind of

In [1]:
from vigenere import VigenereCipher
import time

In [2]:
vc = VigenereCipher()

In [3]:
start = time.time()
analysis_standard = vc.narrow_down(n=3)

end = time.time()
print(f"elapsed time : {end-start}")

elapsed time : 0.015998125076293945


In [15]:
codes = []
for start in range(4, 26):
    to_be_compared = vc.narrow_down(n=3, start=start)
    for code in to_be_compared:
        if code in analysis_standard:
            codes.append((code, start-1))
            print(f"code : {code}  expected_length : {start}")

code : 03 61 94 /22  expected_length : 24
code : 03 61 76 /22  expected_length : 24
code : 52 19 26 /6  expected_length : 25


In [16]:
start = time.time()
analysis_standard = vc.narrow_down(n=5)

end = time.time()
print(f"elapsed time : {end-start}")

elapsed time : 0.16300010681152344


In [17]:
codes = []
for start in range(4, 26):
    to_be_compared = vc.narrow_down(n=5, start=start)
    for code in to_be_compared:
        if code in analysis_standard:
            codes.append((code, start-1))
            print(f"code : {code}  expected_length : {start}")

code : 52 19 26 00 02 /6  expected_length : 25
code : 52 19 26 28 02 /6  expected_length : 25


## 2. building up the key

* offset 6 is known, it's much easier to build up.
* I'll build up using quadgram
* I'll try to figure out till the 23rd key

In [3]:
gram_length = 4
key_length_to_know = 23
iteration = key_length_to_know - gram_length

quad_key_at_start_n = {}
for i in range(iteration):
    quad_key_at_start_n[i] = []

for start in range(iteration):
    standard = vc.narrow_down_offset(n=4, start=start, offset=6)
    to_be_compared = vc.narrow_down_offset(n=4, start=start+25, offset=6)
    for key in to_be_compared:
        if key in standard:
            quad_key_at_start_n[start].append(key)

In [4]:
quad_key_at_start_n

{0: ['52 19 26 28', '52 19 26 00'],
 1: ['19 26 28 02', '19 26 00 02'],
 2: ['26 00 02 17', '26 28 02 17'],
 3: ['00 02 17 49', '28 02 17 49'],
 4: ['02 17 49 65', '02 17 49 38'],
 5: ['17 49 65 63', '17 49 38 87', '17 49 38 63', '17 49 65 87'],
 6: ['49 38 63 08', '49 65 87 08', '49 38 87 08', '49 65 63 08'],
 7: ['65 63 08 94', '65 87 08 94', '38 87 08 94', '38 63 08 94'],
 8: ['87 08 94 70', '63 08 94 70'],
 9: ['08 94 70 92'],
 10: ['94 70 92 88'],
 11: ['70 92 88 27'],
 12: ['92 88 27 63'],
 13: ['88 27 63 39'],
 14: ['27 63 39 35'],
 15: ['63 39 35 30'],
 16: ['39 35 30 68',
  '39 35 30 96',
  '39 35 30 17',
  '39 35 30 31',
  '39 35 30 42'],
 17: ['35 30 42 05',
  '35 30 17 86',
  '35 30 68 05',
  '35 30 96 05',
  '35 30 17 05',
  '35 30 31 05',
  '35 30 31 86',
  '35 30 42 86',
  '35 30 68 86',
  '35 30 96 86'],
 18: ['30 17 05 42',
  '30 17 05 98',
  '30 96 05 98',
  '30 17 05 35',
  '30 96 05 35',
  '30 68 05 42',
  '30 42 86 98',
  '30 31 05 35',
  '30 42 05 40',
  '30 31 05

In [5]:
def build_up_quad_keys(key_dict):
    build_up = key_dict[0]
    for i in range(1, len(key_dict)):
        new = []
        next_quads = key_dict[i]
        for quad in next_quads:
            for building_key in build_up:
                if building_key[-8:] == quad[:8]:
                    new.append(building_key + quad[-3:])
        build_up = new
    return build_up

In [6]:
built_keys = build_up_quad_keys(quad_key_at_start_n)

In [7]:
built_keys

['52 19 26 00 02 17 49 65 87 08 94 70 92 88 27 63 39 35 30 17 05 42',
 '52 19 26 28 02 17 49 65 87 08 94 70 92 88 27 63 39 35 30 17 05 42',
 '52 19 26 00 02 17 49 38 87 08 94 70 92 88 27 63 39 35 30 17 05 42',
 '52 19 26 28 02 17 49 38 87 08 94 70 92 88 27 63 39 35 30 17 05 42',
 '52 19 26 00 02 17 49 65 63 08 94 70 92 88 27 63 39 35 30 17 05 42',
 '52 19 26 28 02 17 49 65 63 08 94 70 92 88 27 63 39 35 30 17 05 42',
 '52 19 26 00 02 17 49 38 63 08 94 70 92 88 27 63 39 35 30 17 05 42',
 '52 19 26 28 02 17 49 38 63 08 94 70 92 88 27 63 39 35 30 17 05 42',
 '52 19 26 00 02 17 49 65 87 08 94 70 92 88 27 63 39 35 30 17 05 98',
 '52 19 26 28 02 17 49 65 87 08 94 70 92 88 27 63 39 35 30 17 05 98',
 '52 19 26 00 02 17 49 38 87 08 94 70 92 88 27 63 39 35 30 17 05 98',
 '52 19 26 28 02 17 49 38 87 08 94 70 92 88 27 63 39 35 30 17 05 98',
 '52 19 26 00 02 17 49 65 63 08 94 70 92 88 27 63 39 35 30 17 05 98',
 '52 19 26 28 02 17 49 65 63 08 94 70 92 88 27 63 39 35 30 17 05 98',
 '52 19 26 00 02 17 

In [18]:
last_three = vc.narrow_down_offset(n=3, start=22, offset=6)
print(last_three)

{'17 78 60', '76 69 18', '17 69 60', '34 78 60', '04 78 18', '24 78 48', '17 78 18', '25 69 48', '34 69 12', '34 78 65', '76 69 65', '76 69 12', '17 78 12', '24 69 60', '24 78 18', '24 69 12', '76 69 48', '17 78 65', '76 78 65', '04 69 12', '25 69 65', '34 69 60', '76 69 60', '04 78 12', '17 78 48', '25 69 18', '25 78 12', '25 78 48', '34 78 18', '34 69 65', '04 69 18', '17 69 18', '17 69 48', '24 69 65', '76 78 12', '24 78 12', '25 78 18', '24 78 60', '25 69 12', '34 69 18', '04 78 48', '04 69 48', '17 69 65', '25 78 60', '04 69 65', '34 78 12', '24 69 18', '76 78 60', '76 78 18', '25 78 65', '76 78 48', '04 78 60', '24 78 65', '25 69 60', '34 78 48', '24 69 48', '34 69 48', '17 69 12', '04 69 60', '04 78 65'}


## 3. Decryption

* Look for plausible one
* Take a look at the second block for now

In [20]:
block3 = {}
for i in range(len(built_keys)):
    plaintext = vc.decrypt(keys=built_keys[i].split(), offset=6, block=3, block_size=25)
    block3[i] = plaintext
    print(f"{i} : {plaintext}")

0 : INFORMAIIONSECURITYERE
1 : INFORMAIIONSECURITYERE
2 : INFORMATIONSECURITYERE
3 : INFORMATIONSECURITYERE
4 : INFORMAIIONSECURITYERE
5 : INFORMAIIONSECURITYERE
6 : INFORMATIONSECURITYERE
7 : INFORMATIONSECURITYERE
8 : INFORMAIIONSECURITYERE
9 : INFORMAIIONSECURITYERE
10 : INFORMATIONSECURITYERE
11 : INFORMATIONSECURITYERE
12 : INFORMAIIONSECURITYERE
13 : INFORMAIIONSECURITYERE
14 : INFORMATIONSECURITYERE
15 : INFORMATIONSECURITYERE
16 : INFORMAIIONSECURITYPRE
17 : INFORMAIIONSECURITYPRE
18 : INFORMATIONSECURITYPRE
19 : INFORMATIONSECURITYPRE
20 : INFORMAIIONSECURITYPRE
21 : INFORMAIIONSECURITYPRE
22 : INFORMATIONSECURITYPRE
23 : INFORMATIONSECURITYPRE
24 : INFORMAIIONSECURITYERE
25 : INFORMAIIONSECURITYERE
26 : INFORMATIONSECURITYERE
27 : INFORMATIONSECURITYERE
28 : INFORMAIIONSECURITYERE
29 : INFORMAIIONSECURITYERE
30 : INFORMATIONSECURITYERE
31 : INFORMATIONSECURITYERE
32 : INFORMAIIONSECURITYPRE
33 : INFORMAIIONSECURITYPRE
34 : INFORMATIONSECURITYPRE
35 : INFORMATIONSECURITYPRE
36

In [21]:
block4 = {}
for i in range(len(built_keys)):
    plaintext = vc.decrypt(keys=built_keys[i].split(), offset=6, block=4, block_size=25)
    block4[i] = plaintext
    print(f"{i} : {plaintext}")

0 : MOSTPOPOLARKEYWORDSWOC
1 : MOSTPOPOLARKEYWORDSWOC
2 : MOSTPOPULARKEYWORDSWOC
3 : MOSTPOPULARKEYWORDSWOC
4 : MOSTPOPOSARKEYWORDSWOC
5 : MOSTPOPOSARKEYWORDSWOC
6 : MOSTPOPUSARKEYWORDSWOC
7 : MOSTPOPUSARKEYWORDSWOC
8 : MOSTPOPOLARKEYWORDSWOJ
9 : MOSTPOPOLARKEYWORDSWOJ
10 : MOSTPOPULARKEYWORDSWOJ
11 : MOSTPOPULARKEYWORDSWOJ
12 : MOSTPOPOSARKEYWORDSWOJ
13 : MOSTPOPOSARKEYWORDSWOJ
14 : MOSTPOPUSARKEYWORDSWOJ
15 : MOSTPOPUSARKEYWORDSWOJ
16 : MOSTPOPOLARKEYWORDSFOJ
17 : MOSTPOPOLARKEYWORDSFOJ
18 : MOSTPOPULARKEYWORDSFOJ
19 : MOSTPOPULARKEYWORDSFOJ
20 : MOSTPOPOSARKEYWORDSFOJ
21 : MOSTPOPOSARKEYWORDSFOJ
22 : MOSTPOPUSARKEYWORDSFOJ
23 : MOSTPOPUSARKEYWORDSFOJ
24 : MOSTPOPOLARKEYWORDSWOO
25 : MOSTPOPOLARKEYWORDSWOO
26 : MOSTPOPULARKEYWORDSWOO
27 : MOSTPOPULARKEYWORDSWOO
28 : MOSTPOPOSARKEYWORDSWOO
29 : MOSTPOPOSARKEYWORDSWOO
30 : MOSTPOPUSARKEYWORDSWOO
31 : MOSTPOPUSARKEYWORDSWOO
32 : MOSTPOPOLARKEYWORDSFOO
33 : MOSTPOPOLARKEYWORDSFOO
34 : MOSTPOPULARKEYWORDSFOO
35 : MOSTPOPULARKEYWORDSFOO
36

In [22]:
block5 = {}
for i in range(len(built_keys)):
    plaintext = vc.decrypt(keys=built_keys[i].split(), offset=6, block=5, block_size=25)
    block5[i] = plaintext
    print(f"{i} : {plaintext}")

0 : TETCENIRELOOKINGPLABNT
1 : TETHENIRELOOKINGPLABNT
2 : TETCENICELOOKINGPLABNT
3 : TETHENICELOOKINGPLABNT
4 : TETCENIRELOOKINGPLABNT
5 : TETHENIRELOOKINGPLABNT
6 : TETCENICELOOKINGPLABNT
7 : TETHENICELOOKINGPLABNT
8 : TETCENIRELOOKINGPLABNF
9 : TETHENIRELOOKINGPLABNF
10 : TETCENICELOOKINGPLABNF
11 : TETHENICELOOKINGPLABNF
12 : TETCENIRELOOKINGPLABNF
13 : TETHENIRELOOKINGPLABNF
14 : TETCENICELOOKINGPLABNF
15 : TETHENICELOOKINGPLABNF
16 : TETCENIRELOOKINGPLALNF
17 : TETHENIRELOOKINGPLALNF
18 : TETCENICELOOKINGPLALNF
19 : TETHENICELOOKINGPLALNF
20 : TETCENIRELOOKINGPLALNF
21 : TETHENIRELOOKINGPLALNF
22 : TETCENICELOOKINGPLALNF
23 : TETHENICELOOKINGPLALNF
24 : TETCENIRELOOKINGPLABNV
25 : TETHENIRELOOKINGPLABNV
26 : TETCENICELOOKINGPLABNV
27 : TETHENICELOOKINGPLABNV
28 : TETCENIRELOOKINGPLABNV
29 : TETHENIRELOOKINGPLABNV
30 : TETCENICELOOKINGPLABNV
31 : TETHENICELOOKINGPLABNV
32 : TETCENIRELOOKINGPLALNV
33 : TETHENIRELOOKINGPLALNV
34 : TETCENICELOOKINGPLALNV
35 : TETHENICELOOKINGPLALNV
36

In [24]:
block3[75] + "___" + block4[75] + "___" + block5[75]

'INFORMATIONSECURITYARE___MOSTPOPULARKEYWORDSTOC___TETHENICELOOKINGPLAINT'

In [25]:
built_keys[75]

'52 19 26 28 02 17 49 38 87 08 94 70 92 88 27 63 39 35 30 31 05 42'

In [27]:
final = []
for key in last_three:
    final.append(built_keys[75] + ' ' + key)

In [28]:
final

['52 19 26 28 02 17 49 38 87 08 94 70 92 88 27 63 39 35 30 31 05 42 17 78 60',
 '52 19 26 28 02 17 49 38 87 08 94 70 92 88 27 63 39 35 30 31 05 42 76 69 18',
 '52 19 26 28 02 17 49 38 87 08 94 70 92 88 27 63 39 35 30 31 05 42 17 69 60',
 '52 19 26 28 02 17 49 38 87 08 94 70 92 88 27 63 39 35 30 31 05 42 34 78 60',
 '52 19 26 28 02 17 49 38 87 08 94 70 92 88 27 63 39 35 30 31 05 42 04 78 18',
 '52 19 26 28 02 17 49 38 87 08 94 70 92 88 27 63 39 35 30 31 05 42 24 78 48',
 '52 19 26 28 02 17 49 38 87 08 94 70 92 88 27 63 39 35 30 31 05 42 17 78 18',
 '52 19 26 28 02 17 49 38 87 08 94 70 92 88 27 63 39 35 30 31 05 42 25 69 48',
 '52 19 26 28 02 17 49 38 87 08 94 70 92 88 27 63 39 35 30 31 05 42 34 69 12',
 '52 19 26 28 02 17 49 38 87 08 94 70 92 88 27 63 39 35 30 31 05 42 34 78 65',
 '52 19 26 28 02 17 49 38 87 08 94 70 92 88 27 63 39 35 30 31 05 42 76 69 65',
 '52 19 26 28 02 17 49 38 87 08 94 70 92 88 27 63 39 35 30 31 05 42 76 69 12',
 '52 19 26 28 02 17 49 38 87 08 94 70 92 88 27 63 39

In [29]:
block5 = {}
for i in range(len(final)):
    plaintext = vc.decrypt(keys=final[i].split(), offset=6, block=5, block_size=25)
    block5[i] = plaintext
    print(f"{i} : {plaintext}")

0 : TETHENICELOOKINGPLAINTTXT
1 : TETHENICELOOKINGPLAINTVDU
2 : TETHENICELOOKINGPLAINTTDT
3 : TETHENICELOOKINGPLAINTEXT
4 : TETHENICELOOKINGPLAINTYXU
5 : TETHENICELOOKINGPLAINTQXK
6 : TETHENICELOOKINGPLAINTTXU
7 : TETHENICELOOKINGPLAINTIDK
8 : TETHENICELOOKINGPLAINTEDR
9 : TETHENICELOOKINGPLAINTEXP
10 : TETHENICELOOKINGPLAINTVDP
11 : TETHENICELOOKINGPLAINTVDR
12 : TETHENICELOOKINGPLAINTTXR
13 : TETHENICELOOKINGPLAINTQDT
14 : TETHENICELOOKINGPLAINTQXU
15 : TETHENICELOOKINGPLAINTQDR
16 : TETHENICELOOKINGPLAINTVDK
17 : TETHENICELOOKINGPLAINTTXP
18 : TETHENICELOOKINGPLAINTVXP
19 : TETHENICELOOKINGPLAINTYDR
20 : TETHENICELOOKINGPLAINTIDP
21 : TETHENICELOOKINGPLAINTEDT
22 : TETHENICELOOKINGPLAINTVDT
23 : TETHENICELOOKINGPLAINTYXR
24 : TETHENICELOOKINGPLAINTTXK
25 : TETHENICELOOKINGPLAINTIDU
26 : TETHENICELOOKINGPLAINTIXR
27 : TETHENICELOOKINGPLAINTIXK
28 : TETHENICELOOKINGPLAINTEXU
29 : TETHENICELOOKINGPLAINTEDP
30 : TETHENICELOOKINGPLAINTYDU
31 : TETHENICELOOKINGPLAINTTDU
32 : TETHENICELOOK

3 is the thing!

In [30]:
final_key = final[3]
final_key

'52 19 26 28 02 17 49 38 87 08 94 70 92 88 27 63 39 35 30 31 05 42 34 78 60'

## 3 Final Decryption

In [32]:
original = ""
for block in range(1, 6):
    plaintext = vc.decrypt(keys=final_key.split(), offset=6, block=block, block_size=25)
    original += plaintext
print(original)

THISCIPHERWASWIDELYUSEDBECAUSEOFSIMPLESTRUCTURETHEINFORMATIONSECURITYARETHEMOSTPOPULARKEYWORDSTOCREATETHENICELOOKINGPLAINTEXT
